<a href="https://colab.research.google.com/github/thakerpragnesh/DeepLearning/blob/master/10_ComputeScore_CustomPruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Important Imports

#### Following Cell facilitate to read files from drive and and help to read dataset

In [1]:
#import basic library for some basic function
import numpy as np
# import library to perform file operation
import os #use to access the files 
import tarfile # use to extract dataset from zip files
import sys
import zipfile

#### Torch Library provides facilities to create networl architechture and write farword and backwor phase od neural network

In [2]:
#import torch library to build neural network
import torch  # Elementory function of tensor is define in torch package
import torch.nn as nn # Several layer architectur is define here
import torch.nn.functional as F # loss function and activation function

#### torchvission provides facilities to access image dataset

In [3]:
"""
Computer vision is one of the most important application and thus lots 
of deplopment in the and torch.vision provides many facilities that can 
be use to imporve model such as data augmentation, reading data batchwise, 
suffling data before each epoch and many more
"""
# import torch library related to image data processing
import torchvision # provides facilities to access image dataset
from torchvision.datasets.utils import download_url 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import datasets, models, transforms

# Load the dataset


In [4]:
import os
import torch
def ensure_dir(dir_path):
    directory = os.path.dirname(dir_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [5]:
######################## Input ###############################
DatasetLoc = '/home/pragnesh/Dataset/'
SelDataSet = 'IntelIC'
trainDir = 'train'
testDir = 'test'
data_dir = DatasetLoc+SelDataSet
zipFile = False

In [6]:
##################### Output ##################################
#/home3/pragnesh/Dataset/Intel_Image_Classifacation_v2/
SelectOutLoc = DatasetLoc+"Intel_Image_Classifacation_v2/"  
LogLoc =   SelectOutLoc+"Logs/"
outfile = LogLoc+"FinalOutv2.log"
logFile = LogLoc+"ConvModelv2.log"

In [7]:
###################### Model #################################
ifLoadModel = False
modelname ='vgg16' # vgg11 vgg13 vgg16,  resnet18,  savedmodel
ifTransferLearning = True
NumberOfClass = 6
ModelLoc = SelectOutLoc+"Model/"
SavePath = SelectOutLoc+'Model/VGG_IntelIC_v1-'+modelname
LoadPath = '/content/drive/MyDrive/Model/IntelIC/VGG16IntelIC'#SelectOutLoc+'Model/VGG_IntelIC_v1-'+modelname

In [8]:
##################### Hyper Parameter #########################
"""
***************************************************************
We are using one cycle fit function in which learning rate start with 1/10th 
of selected maximum learning rate and increase learning rate from min to max
in 1st phase and then decrease from max to min in 2nd phase
***************************************************************
Set all the Hyper Parameter Such as 
1. Learning Rate to control step size
2. grad_clip to control the maximum value of gradient
3. weight decay to control L2 regularization
4. L1 to control L1 regularization
5. opt_func to select optimization function
***************************************************************
"""
max_lr = 1e-3
epochs = 1
grad_clip = 0.2 
weight_decay = 1e-4 
L1 = 1e-5
opt_func = torch.optim.Adam
MODEL_NAME = f"VGG_Net-\t MLR-{max_lr}-GC{grad_clip}-WD-{weight_decay}-L1-{L1}"

#################################################################

In [9]:
ensure_dir(SelectOutLoc)
ensure_dir(ModelLoc)
ensure_dir(LogLoc)

In [10]:
#Data Prepration
"""
Based on the image size of the dataset choose apropriate values of the color channel and Image Size

Here we can define path to a folder where we can keep all the dataset. 
In the following we are using the zip files. Originally dataset should 
be in the following format DataSetName is parent folder and it should 
contain train and test folder. train and test folder should contain 
folder for each category and images of respective category should be in 
the respective category folder
"""
######################### Data Loading #########################################
if zipFile == True:
  fullpath = data_dir+'.zip'
  zip_ref = zipfile.ZipFile(fullpath, 'r') #Opens the zip file in read mode
  zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
  data_dir = '/tmp/IntelIC'
  testDir ='val'
  zip_ref.close()


### Create batches of the data and perform transformation

In [11]:
"""
Choose an apropriate batch size that can be loaded in the current 
enviroment without crashing and also do not choose too big batch even 
if dataset is small because it leads to very few updates per epoch
"""
#################### Create Batch Of Dataset and do data augmentation ###########
bs = 16
ImageSize = 224

"""
Data Augmentaion generally help in reducing overfitting error during 
trainng process and thus we are performing randon horizontal flip and 
random crop during training but during validation as no training happens 
we dont perform data augmentation
"""
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    trainDir: transforms.Compose([
        transforms.RandomResizedCrop(ImageSize),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    testDir: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in [trainDir, testDir]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,
                                             shuffle=True, num_workers=1)
              for x in [trainDir, testDir]}

dataset_sizes = {x: len(image_datasets[x]) for x in [trainDir, testDir]}
class_names = image_datasets[trainDir].classes

# Define Training Process

#### Define functions to facilitate training process

In [12]:
"""#Training Process
Below code will work as a base function and provide all the important 
function like compute loss, accuracy and print result in a perticular 
formate afte each epoch. Funvtion are as follow
1. Accuracy : Computer accuracy in evalutaion mode of pytorch on given dataset for given model
2. compute_batch_loss : Compute batch loss and append the loss in the list of batch loss.
3. compute_batch_loss_acc : Compute batch loss, batch accuracy and append the loss in the list of batch loss.
4. accumulate_batch_loss_acc: Accumulate loss from the list of batch and acccuraly loss.
5. Epoch end to print the output after every epoch in proper format
"""
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1) 		# get the prediction vector
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# Compute loss of the given batch and return it
def compute_batch_loss(newmodel, batch_X,batch_y):
  images = batch_X.to(device)
  labels = batch_y.to(device)
  out = newmodel(images)                  		# Generate predictions
  loss = F.cross_entropy(out, labels) 			# Calculate loss
  return loss

# Computes loss and accuracy of the given batch(Used in validation)
def compute_batch_loss_acc(newmodel, batch_X,batch_y):
    images = batch_X.to(device)
    labels = batch_y.to(device)
    out = newmodel(images)                    	# Generate predictionsin_features=4096
    loss = F.cross_entropy(out, labels)   		# Calculate loss
    acc = accuracy(out, labels)           		# Calculate accuracy
    return {'val_loss': loss, 'val_acc': acc}

# At the end of epoch accumulate all batch loss and batch accueacy    
def accumulate_batch_loss_acc(outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

def epoch_end(epoch, result):
  # Print in given format 
  # Epoch [0], last_lr: 0.00278, train_loss: 1.2862, val_loss: 1.2110, val_acc: 0.6135
  strResult = "Epoch [{}], last_lr: {:.6f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
      epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc'])
  #print(strResult)
  return strResult

#### Define actual training process

In [13]:
"""## Define Training 
Here we will evalute our model after each epoch on validation dataset using evalute method
get_lr method returnd last learning rate used in the training
Here we are using one fit cycle method in which we specify the max learning rate and learning 
rate start from 1/10th value of max_lr and slowly increases the value to max_lr for 40% of updates 
then decreases to its initial value for 40% updates and then further decreases to 1/100th of max_lr 
value to perform final fine tuning.
"""
# evalute model on given dataset using given data loader
@torch.no_grad()
# evalute model on given dataset using given data loader
def evaluate(model, data_loader):
    model.eval()
    with torch.no_grad():
      for batch_X, batch_y in data_loader:
        outputs = [compute_batch_loss_acc(model,batch_X,batch_y)]
      return accumulate_batch_loss_acc(outputs)

# Use special scheduler to change the value of learning rate
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# epoch=8, max_lr=.01, weight_decay(L2-Regu parametr)=.0001,opt_func=Adam

######### Main Function To Implement Training #################
def fit_one_cycle(ModelName,epochs, max_lr, model, 
                  weight_decay=0, L1=0,grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    # Set up cutom optimizer here we will use one cycle scheduler with max learning
    # rate given by max_lr, default optimizer is SGD but we will use ADAM, and 
    # L2 Regularization using weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(dataloaders[trainDir]))
    print("Training Starts")
    with open(logFile, "a") as f:
      for epoch in range(epochs):
          # Training Phase 
          model.train()  #######################
          train_losses = []
          lrs = []
          #for batch in train_loader:
          for batch_X, batch_y in dataloaders[trainDir]:
              # computer the training loss of current batch
              loss = compute_batch_loss(model,batch_X,batch_y)
              l1_crit = nn.L1Loss()
              reg_loss = 0
              for param in model.parameters():
                reg_loss += l1_crit(param,target=torch.zeros_like(param))
              loss += L1*reg_loss 
              
              train_losses.append(loss)
              loss.backward() # compute the gradient of all weights
              # Clip the gradient value to maximum allowed grad_clip value
              if grad_clip: 
                  nn.utils.clip_grad_value_(model.parameters(), grad_clip)
              optimizer.step() # Updates weights 
              # pytorch by default accumulate grade history and if we dont want it
              # we should make all previous grade value equals to zero
              optimizer.zero_grad() 
              # Record & update learning rate
              lrs.append(get_lr(optimizer))
              sched.step() # Update the learning rate
              # Compute Validation Loss and Valodation Accuracy
              result = evaluate(model, dataloaders[testDir])
              # Compute Train Loss of whole epoch i.e mean of loss of batch 
              result['train_loss'] = torch.stack(train_losses).mean().item()
              # Observe how learning rate is change by schedular
              result['lrs'] = lrs
              # print the observation of each epoch in a proper format
          
          #strResult = "Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, 
            #val_acc: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc'])
          strResult = epoch_end(epoch, result) 
          
          f.write(f"{ModelName}-\t{strResult}\n")
          print(strResult)
          history.append(result) # append tupple result with val_acc, vall_loss, and trin_loss
        
    return history

#### check for cuda device

In [14]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
"""Check if Cuda GPU is available"""
#check for CUDA enabled GPU card
def getDeviceType():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')
device = getDeviceType()
print(device)

cpu


# Define vggnet architecture

In [15]:
###Define a VggNet Architecture
from typing import Union, List, Dict, Any, cast
class VGG(nn.Module):
    def __init__(
        self, features: nn.Module, num_classes: int = 10, init_weights: bool = True, dropout: float = 0.5
    ) -> None:
        super().__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    
def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
    #    print(count)
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    #    count+=1
    return nn.Sequential(*layers)


def _vgg(VggFeatureList, batch_norm: bool, progress: bool) -> VGG:
    
    #Calling the constructer here
    feature = make_layers(VggFeatureList, batch_norm=batch_norm)
    model = VGG( feature )
    return model

In [16]:
vgg16Feature=[64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
vgg16Net = _vgg( vgg16Feature, batch_norm=False, progress=False)
print(vgg16Net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Select and load apropriate model

In [ ]:
# In[11]: Load the apropiate model for training
############################# Use Pre-Train Model for Transfer Learning #####################################
#################### Download Pretrain VGGNet ################
def loadPretrainModel(modelname,pretrain: bool = False):
    if modelname == 'vgg16' or modelname == 'vgg13' or modelname == 'vgg11':
        if modelname == 'vgg11':
            newModel = torchvision.models.vgg11(pretrained=False)
        if modelname == 'vgg13':
            newModel = torchvision.models.vgg13(pretrained=False)
        if modelname == 'vgg16':
            newModel = torchvision.models.vgg16(pretrained=False)    
        if ifTransferLearning:
            for param in newModel.parameters():
                param.requires_grad = False
        #Need to change the below code if we choose different model
        print(newModel.classifier[6])
        num_ftrs = newModel.classifier[6].in_features
        # Here the size of each output sample is set to 10.
        # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
        newModel.classifier[6] = nn.Linear(num_ftrs, NumberOfClass)
        newModel = newModel.to(device)
        return newModel
    if modelname == 'vgg16bn' or modelname == 'vgg13bn' or modelname == 'vgg11bn':
        if modelname == 'vgg11bn':
            newModel = torchvision.models.vgg11_bn(pretrained=False)
        if modelname == 'vgg13bn':
            newModel = torchvision.models.vgg13_bn(pretrained=False)
        if modelname == 'vgg16bn':
            newModel = torchvision.models.vgg16_bn(pretrained=False)    
        if ifTransferLearning:
            for param in newModel.parameters():
                param.requires_grad = False
        #Need to change the below code if we choose different model
        print(newModel.classifier[6])
        num_ftrs = newModel.classifier[6].in_features
        # Here the size of each output sample is set to 10.
        # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
        newModel.classifier[6] = nn.Linear(num_ftrs, NumberOfClass)
        newModel = newModel.to(device)
        return newModel
    ##################### Download Pretrain ResNet 18 ############################
    if modelname == 'resnet18':
        newModel = torchvision.models.resnet18(pretrained=False)
        print(newModel.fc)
        for param in newModel.parameters():
            param.requires_grad = False
        #print(newModel)
        #Need to change the below code if we choose different model
        num_ftrs = newModel.fc.in_features
        # Here the size of each output sample is set to 10.
        # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
        newModel.fc = nn.Linear(num_ftrs, 10)
        newModel = newModel.to(device)
        return newModel

################# Load Stored Trained Model #####################################
def loadSavedModel(LoadPath,device=torch.device('cpu')):
    if device== torch.device('cpu'):
        newModel = torch.load(LoadPath, map_location=torch.device('cpu'))
    else:
        newModel = torch.load(LoadPath, map_location=torch.device('cuda'))

def freezeFeature(modelname,newModel):
    count=0
    if modelname == 'VGG16':
        for param in newModel.parameters():
            if count in (26,28,30):
              param.requires_grad=True
            else:
              param.requires_grad=False
                
    if modelname == 'VGG13':
        for param in newModel.parameters():
            if count in (20,22,24):
              param.requires_grad=True
            else:
              param.requires_grad=False
            
    if modelname == 'VGG11':
        for param in newModel.parameters():
            if count in (16,18,20):
              param.requires_grad=True
            else:
              param.requires_grad=False
    
def freeze(modelname,newModel):
    count=0
    if modelname == 'VGG16':
        for param in newModel.parameters():
            if count == 30:
              param.requires_grad=True
            else:
              param.requires_grad=False
                
    if modelname == 'VGG13':
        for param in newModel.parameters():
            if count == 24:
              param.requires_grad=True
            else:
              param.requires_grad=False
            
    if modelname == 'VGG11':
        for param in newModel.parameters():
            if count == 20:
              param.requires_grad=True
            else:
              param.requires_grad=False
            
def unFreaze(newModel):
    for param in newModel.parameters():
        param.requires_grad=True

In [ ]:
if ifLoadModel:
    newModel = loadSavedModel(LoadPath)
    freezeFeature(modelname,newModel)
else:
    newModel = loadPretrainModel(modelname)

In [ ]:
print(newModel)


# Perform the training
Start the training using one fit cycle algorithms where learning rate start with one tenth of provided max_lr and then increase it value till that point and then decrease onword and for last few epoch learning rate furthe decrease
Pass the hyperparameter for training of the model and start training process for set number of epcoh

In [ ]:
# %%time
historylast=[]
historylast += fit_one_cycle(MODEL_NAME,epochs, max_lr, newModel,  
                              grad_clip=grad_clip, 
                              weight_decay=weight_decay, L1=L1,
                              opt_func=opt_func
                              )
################################################ Evalute the Training Process by Plotting Graph ###################################
torch.save(newModel, SavePath)

## Evalute the model

In [ ]:
"""##Define method to compute accuracy for a given model on given dataset"""
def accuraciesTotal(newModel, data_loader):
  with torch.no_grad():
    acc = []
    for batch in data_loader:
        images, label = batch
        images, labels = batch[0].to(device), batch[1].to(device)
        out = newModel(images)
        acc.append(accuracy(out, labels))
    return torch.mean(torch.stack(acc))

#### Prepare dataset for evaluation of the model

In [ ]:
### Prepare the data loader for inference. During the 
### inference we want to perform same tranformation on test and train dataset"""
image_datasets_eval = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[testDir])
                  for x in [trainDir, testDir]}
dataloaders_eval = {x: torch.utils.data.DataLoader(image_datasets_eval[x], batch_size=bs,
                                             shuffle=False, num_workers=1)
                  for x in [trainDir, testDir]}

#### Evalute the model

In [ ]:
# In[17]: Evalutute train and test error
trainacc = 0
testacc = 0
trainacc = accuraciesTotal(newModel,dataloaders_eval[trainDir])
testacc = accuraciesTotal(newModel,dataloaders_eval[testDir])

with open(outfile,'a') as f:
    f.write(f"Train Accuracy :  {trainacc}\n Test Accuracy  :  {testacc}")

# Pruning Process


In the below code we create a list module with candidate layer to prune with their respective probability

In [ ]:
import my_utils.initialize_pruning as ip

In [ ]:
'''
Get the indices of the different model that gives parameter to be prune
'''
        
Module = ip.getPruneModule(newModel)
blockList = ip.createBlockList(newModel)
featureList = ip.createFeatureList(newModel)
pr = ip.getPruneCount(module=Module,blocks=blockList,maxpr=.1)

# for i in range(len(pr)):
#     size = Module[i]._parameters['weight'].shape
#     c = int(round(size[0]*pr[i]))
#     prune_count.append(c)
#     print('%.4f  '%pr[i],end="")

print("Block List   =",blockList)
print("Feature List =",featureList)
print("Prune Count  =",pr)
for i in range(len(Module)):
        print(f"{i} : {Module[i]._parameters['weight'].shape}")

Compute the distance betweeen each pair of kernal working on same input channel

In [ ]:
#t = Tensor to be prune, n is ln normalization, dim dimension over which we want to perform 
def _compute_distance_score(t, n=1, dim_to_keep=[0,1],threshold=1):
        # dims = all axes, except for the one identified by `dim`        
        dim_to_prune = list(range(t.dim()))   #initially it has all dims
        #remove dim which we want to keep from dimstoprune
        for i in range(len(dim_to_keep)):   
            dim_to_prune.remove(dim_to_keep[i])
        
        size = t.shape
        print(f"\nShape of the tensor: {size}")
        print(f"Print the Dims we want to keep: {dim_to_keep}")
        
        module_buffer = torch.zeros_like(t)
                
        #shape of norm should be equal to multiplication of dim to keep values
        norm = torch.norm(t, p=n, dim=dim_to_prune)
        print(f"norm shape = {norm.shape}")
        size = t.shape
        print("Number Of Features Map in current  layer l     =",size[0])
        print("Number Of Features Map in previous layer (l-1) =",size[1])
        
        for i in range(size[0]):
            for j in range(size[1]):
                module_buffer[i][j] = t[i][j]/norm[i][j]
        
        dist = torch.zeros(size[1],size[0],size[0])
        
        channelList = []
        for j in range(size[1]):
            idxtupple = []
            print('.',end='')
            for i1 in range(size[0]):
                for i2 in range((i1+1),size[0]):
                    dist[j][i1][i2] = torch.norm( (module_buffer[i1][j]-module_buffer[i2][j]) ,p=1)
                    dist[j][i2][i1] = dist[j][i1][i2]
                    
                    if dist[j][i1][i2] < threshold:
                        idxtupple.append([j,i1,i2,dist[j][i1][i2]])
            channelList.append(idxtupple)
        return channelList

Compute the Cummulative value of each kernal working on same input channel


In [ ]:
#t = Tensor to be prune, n is ln normalization, dim dimension over which we want to perform 
def _compute_kernal_score(t, n=1, dim_to_keep=[0,1],threshold=1):
        # dims = all axes, except for the one identified by `dim`        
        dim_to_prune = list(range(t.dim()))   #initially it has all dims
        #remove dim which we want to keep from dimstoprune
        for i in range(len(dim_to_keep)):   
            dim_to_prune.remove(dim_to_keep[i])
        
        size = t.shape
        print(size)
        print(dim_to_keep)
        
        module_buffer = torch.zeros_like(t)
                
        #shape of norm should be equal to multiplication of dim to keep values
        norm = torch.norm(t, p=n, dim=dim_to_prune)
        return norm

In [ ]:
channelTuppleList = []
st =0
en = 4
for i in range(st,en):
    channelTuppleList.append(_compute_distance_score(Module[i]._parameters['weight'],threshold=1))
print("\n\n\nHere is the :",len(channelTuppleList))

In [ ]:
for i in range(len(channelTuppleList)):
    for j in range(len(channelTuppleList[i])):
        print(f"Number of filter in Channel [{i}][{j}] below threshold: {len(channelTuppleList[i][j])}")

In [ ]:
def displayLayer(channelTupple):
    for i in range(len(channelTupple)):
        for j in range(len(channelTupple[i])):
            if j%3==0:
                print()
        print(channelTupple[i][j],'\t',end='')  

In [ ]:
def makeChannelList(Module,startIdx,endIdx,thresh_hold):
    channelTuppleList = []
    for i in range(st,en):
        channelTuppleList.append(_compute_distance_score(Module[i]._parameters['weight'],threshold=thresh_hold))
    print("\n\n\nHere is the :",len(channelTuppleList))
    return channelTuppleList

In [ ]:
def sortKernal(kernalList):
  for i in range(len(kernalList)):
    iListLen = len(kernalList[i])
    #print(f'lemgth of list {i} ={iListLen}')
    for j in range(iListLen):
      for k in range(iListLen-j-1):
        #print(f"Value of i={i}     Value of j={j} Value of k={k}")
        if kernalList[i][k+1][3] < kernalList[i][k][3]:
          kernalList[i][k+1], kernalList[i][k] = kernalList[i][k], kernalList[i][k+1]

In [ ]:
def get_k_element(channel_list,k):
    channel_k_list = []
    for i in range(len(channelTuppleList)):
        for j in range(k):
            channel_k_list.append(channel_list[i][j])
    return channel_k_list

In [ ]:
print("Channe list :",len(channelTuppleList))
for i in range(3):
    print("Number of input channel :",len(channelTuppleList[i]))
    for j in range(3):
        print(f"Number of kernel pair with simirities diff less than threshold {i} :{len(channelTuppleList[i][j])}")
        for k in range(3):
            print("Channe list [] [] [] :",(channelTuppleList[i][j][k]))
        

In [ ]:
for i in range(len(channelTuppleList)):
  sortKernal(channelTuppleList[i])

In [ ]:
newList = []
for i in range(len(channelTuppleList)):
    newList.append(get_k_element(channel_list=channelTuppleList[i],k=3) )
for i in range(len(newList)):
    print(len(newList[i]))

In [ ]:
print(newList)

In [ ]:
for i in range(len(channelTuppleList)):
  print("\n**************************************************************************************************************************")
  displayLayer(channelTuppleList[i])

## Pruning Start From Here

#### Define the Pruning Class and Pruning Function

In [ ]:
import torch.nn.utils.prune as prune
class KernalPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        #mask.view(-1)[::2] = 0
        size = t.shape
        print(size)
        for k1 in range(len(newList[layer_number])):
            for k2 in range(len(newList[layer_number][k1])):
                i= newList[layer_number][k1][k2][1]
                j= newList[layer_number][k1][k2][0]
                #print(f'i={i}   j={j}')
                mask[i][j] = 0
        return mask
def kernal_unstructured(module, name):
    KernalPruningMethod.apply(module, name)
    return module

#### Perform pruning on list of Mudele using custom pruning method

In [ ]:
layer_number = 0
for i in range(3):
    kernal_unstructured(Module[i], name = 'weight')
    layer_number += 1

#### Commit the pruning and remove all buffer associated with it

In [ ]:
for i in range(3):
    prune.remove(Module[i], 'weight')

#### Write down deep Copy Code Which Copy Weights Of The Kernal That copy weights which have non zero weight in kernal